In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.4.1


**Note:** tensorflow_datasets (tfds for short) contains many types of datasets. IMDB reviews is one of them. It contains 50,000 movie reviews that are classified as positive or negative. There are 25,000 samples for training and testing each.

In [22]:
import tensorflow_datasets as tfds
imdb,info = tfds.load("imdb_reviews",with_info=True,as_supervised=True)

**Note:** The labels and reviews are stored as tensors. A tensor can be thought of as a container that has N-dimensional data. Since we need our data to be an array for training, we use numpy to convert the tensors into arrays.

In [24]:
import numpy as np

In [25]:
train_data,test_data= imdb["train"],imdb["test"]

In [32]:
training_sentences,training_labels =[],[]
testing_sentences,testing_labels = [],[]

In [33]:
for s,l in train_data :
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

In [34]:
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [51]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

# Need numpy arrays instead of straight arrays for training:  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [35]:
# Hyperparameters on top for easy modification.
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [36]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [37]:
tokenizer= Tokenizer(num_words = vocab_size,oov_token=oov_tok)

In [38]:
tokenizer.fit_on_texts(training_sentences)

In [39]:
word_index = tokenizer.word_index

In [40]:
sequences = tokenizer.texts_to_sequences(training_sentences)

In [41]:
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

In [42]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

In [43]:
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [46]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

**Note:** If sentences (reviews) are shorter than max_length, they will be padded. If they are longer than max_length, they will be truncated.

**Note:** Remember to fit the tokenizer on the training sentences only! Not the testing sentences.

**Note:** Since the tokenizer was fitted on the training set, you should expect to see a lot more OOV tokens when you use it for the test set.

**Note:** Use reverse_word_index to reverse the keys and values in a dictionary.

**Note:** The get() method takes maximum of two parameters:

1. key - key to be searched in the dictionary
2. value (optional) - Value to be returned if the key is not found. The default value is None.

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3])) # Processed sentence output
print(training_sentences[3]) # Original sentence

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


**Note:** The embedding will be a 2D array of size (No. of features) * (Length of vocabulary). We use max_length to set the same encoding length for all words in the vocabulary.

**Note:** Instead of using Flatten(), which turns all the rows of the array into a single line, you can use Global Average Pooling, which will take the average across all words for each feature. This will return a 1 dimensional vector with the number of elements equal to the number of features. This should make the computations for the Dense layers easier and faster.

In [48]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # Only 2 output classes
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [76]:
weights[0].shape

(10000, 16)

In [ ]:
model.fit(padded,
          training_labels_final,epochs=num_epochs

In [52]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 5ms/step - loss: 0.6069 - accuracy: 0.6345 - val_loss: 0.3518 - val_accuracy: 0.8436
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2355 - accuracy: 0.9106 - val_loss: 0.3782 - val_accuracy: 0.8370
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0870 - accuracy: 0.9797 - val_loss: 0.4695 - val_accuracy: 0.8239
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0216 - accuracy: 0.9972 - val_loss: 0.5361 - val_accuracy: 0.8263
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0057 - accuracy: 0.9996 - val_loss: 0.6034 - val_accuracy: 0.8250
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.6499 - val_accuracy: 0.8264
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 8.9375e-04 - accuracy: 1.0000 - val_loss: 0.6923 - val_accuracy: 0.8272
Ep

**Note:** Training accuracy of 1 from epoch 8 onwards shows we are overfitting!

**Note:** The following code is used to create meta.tsv (actual words) and vecs.tsv (vectors of corresponding words). These are used to visualize the word vectors in the embedding projector at https://projector.tensorflow.org.

In [77]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [81]:
# reverse_word_index
# weights

In [86]:
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    print(word)
    print(embeddings)

<OOV>
[ 0.01427425  0.05788165  0.04453075 -0.11687026  0.16397876 -0.0346463
  0.01320624 -0.03201183 -0.00147527  0.01031678 -0.03498985 -0.0128717
 -0.02241344 -0.07057899 -0.0027815   0.06396645]
the
[ 0.01362592  0.07939275 -0.0152524  -0.04633929  0.08443374 -0.11170074
  0.01931684 -0.00654864  0.0772144   0.00051395 -0.08414985 -0.0023776
 -0.04821678 -0.06941254  0.00136358  0.08548049]
and
[ 0.03924621  0.01031127  0.01111411 -0.03188766  0.01737459 -0.1011737
  0.05879349 -0.02534689  0.01830909  0.04297912 -0.09354249  0.0183557
 -0.05437945 -0.16042717  0.05514784 -0.03872989]
a
[-0.01678534  0.09727963  0.01354177 -0.01227822  0.12010585 -0.016131
  0.03489377  0.01600146  0.07501965 -0.01234785 -0.099245    0.00158142
  0.00802053 -0.07075791  0.02326007  0.04115211]
of
[ 0.01431945  0.04874268  0.0032153   0.01033358  0.07390741 -0.07148011
 -0.02420017 -0.01387657 -0.01151765 -0.00922603 -0.08654186 -0.01936505
  0.03620084 -0.0225769   0.0405318   0.05857238]
to
[ 0.0

year
[ 0.09115139  0.03641043 -0.04515423 -0.00207391  0.0125839   0.00663177
 -0.02240252 -0.13311324  0.02418913  0.03112634  0.03498939  0.15849239
  0.0096375  -0.15078917 -0.09227367 -0.15101288]
sense
[ 0.01330362 -0.10495064  0.06693681 -0.10942656 -0.02754577  0.07802601
  0.02342715 -0.01217149 -0.05535205 -0.07221681  0.02812989 -0.02703954
 -0.03911949  0.2916277  -0.00451182  0.03051185]
reason
[ 0.1663839   0.04738024  0.12409256 -0.09266804 -0.09369265 -0.11459129
  0.02791199  0.06713555 -0.15441194 -0.09127495  0.15147562 -0.02336353
  0.00225348  0.01646038 -0.08132301  0.06215652]
true
[-0.10059714 -0.12120618 -0.15983655  0.04998314  0.1036443  -0.03909939
  0.05537757 -0.1123292   0.01141033  0.18264106  0.14079681 -0.1102217
 -0.13211726  0.01862454  0.14345111 -0.18180819]
everything
[-0.07802547  0.07095088 -0.01560539  0.05852589  0.04707171 -0.00316826
 -0.00397451  0.15107915  0.04494124  0.07011668 -0.01124475  0.07393976
 -0.00472535  0.10205887  0.02135363 

  0.21632794  0.09833787 -0.07678344  0.17158334]
gore
[-4.4137221e-02 -8.3308302e-02  8.6092085e-02 -5.6154102e-02
 -4.5484293e-02 -2.6552944e-02 -1.3956514e-01  2.0035762e-01
 -1.7991102e-01 -2.0870222e-02 -1.0720156e-01  8.5282288e-02
  7.5387225e-02 -7.4347861e-02 -1.1417950e-04  1.0336119e-01]
crap
[ 0.23302832  0.20576039  0.17339587 -0.18190144 -0.16117704 -0.00062527
 -0.19851285 -0.17756742 -0.18039893 -0.23962656  0.03942957 -0.00607163
  0.20233749 -0.15762609  0.01041121  0.20064637]
ago
[ 0.0054306   0.18772236  0.06341669 -0.0392709  -0.03362892  0.13912007
 -0.01920502 -0.1011124   0.12898515  0.04238299 -0.05263614  0.07211707
 -0.08556101  0.03465303  0.01507752  0.00075532]
group
[ 0.00989193  0.04863966  0.07596114 -0.22981727 -0.08263452 -0.09851661
 -0.11035639  0.13321148  0.02808542 -0.03792488  0.02716581  0.04072847
 -0.00964422 -0.09871204 -0.03213188  0.0203655 ]
complete
[ 0.09214564  0.0953112   0.12713885 -0.00503138 -0.1860485  -0.06678435
  0.06541903  0

[-0.04865948 -0.10324675 -0.08817443  0.07216958  0.16689438  0.17939702
  0.05365909  0.15161273  0.05428933  0.05149359 -0.15045536 -0.13370794
 -0.05772902 -0.02626081 -0.01778401  0.01488645]
ask
[-0.0288341  -0.13733587  0.0819288  -0.03585188  0.09194624 -0.08211142
  0.03029152  0.06766372  0.0607478   0.04859209  0.19370395  0.08417457
  0.01200217 -0.09512281  0.10446308  0.06302855]
development
[ 0.18571931  0.1708801   0.08386619 -0.19880114 -0.00606549 -0.0317264
 -0.07361584 -0.03412956 -0.18520354 -0.07044689 -0.01157978  0.14383774
  0.05500527  0.10126504  0.04485074  0.01654887]
features
[-0.04581323  0.03780775  0.02673152  0.07459939 -0.08101641 -0.11565327
  0.14359094  0.10451379  0.00680544 -0.09429367 -0.01926062 -0.04580133
  0.04250016 -0.04456532 -0.07271625  0.04955681]
air
[ 0.01331693  0.11755395  0.02801242 -0.05958875 -0.11407331  0.03003085
 -0.07111218 -0.02165447  0.16218713  0.00836528 -0.08876319  0.37498862
  0.05036377  0.07615622 -0.09721715 -0.11

 -0.12514615  0.00823046  0.2749459  -0.0991986 ]
added
[ 0.03088109  0.00684667  0.02374785 -0.03977139  0.00504058 -0.01952682
  0.0290849   0.14995575 -0.10769682 -0.0553477   0.27864757 -0.08474679
  0.038907   -0.1256188  -0.0688049   0.15317613]
edge
[-0.09553428 -0.17058274 -0.02766595  0.07074387  0.13086218 -0.03100032
 -0.02644028 -0.05930224  0.10234184  0.01743996  0.04259885 -0.09450767
 -0.11245999  0.07157978  0.09054288 -0.06134795]
london
[ 0.08733795  0.08578667  0.09536131 -0.00290342 -0.07277076 -0.06922796
 -0.08083198 -0.06530768 -0.10600848 -0.02862199  0.04425963  0.07692794
  0.13031612 -0.1369533   0.05634403  0.0506056 ]
90
[ 3.66769359e-02  1.16246521e-01 -8.49616434e-03 -6.39887666e-03
  7.23831356e-02  1.23475105e-01 -1.76775843e-01  3.20045054e-02
  1.37484893e-01  3.82063985e-02  4.58568707e-02  2.11995825e-01
  1.51984845e-04 -1.10096447e-01  8.24874546e-03  7.13105372e-04]
purpose
[ 7.3691979e-02  4.5430057e-02  7.2820902e-02 -5.1492430e-02
  9.5774576

...

emphasize
[ 0.10927235  0.1098871   0.05021154 -0.03300733 -0.01343522 -0.09842338
 -0.11723159 -0.18619727 -0.02771258 -0.06490123 -0.13729733  0.03516794
  0.10589543  0.00319734 -0.03361263  0.04489023]
bloodshed
[ 0.10550144  0.02019508  0.03281472 -0.06570876 -0.08473451 -0.02208155
 -0.00837741  0.06490123 -0.10687305 -0.10747415  0.03538898  0.00494951
  0.08759634  0.02217003  0.00158619  0.04343355]
entertains
[-0.16589227 -0.18762518 -0.13526626  0.11754738  0.00404498 -0.11389701
  0.13703397 -0.00053151  0.17444925  0.21466689 -0.05367361 -0.01018898
 -0.24107115 -0.14768694 -0.03872964 -0.20422253]
coal
[-0.05755169 -0.11505347 -0.07290407 -0.01150826  0.25050905 -0.12455652
  0.12675627 -0.00180689  0.01866454  0.03828416  0.11345824 -0.06827512
  0.03173895  0.01441576  0.04878745  0.01406374]
mines
[-0.08169478 -0.07831211 -0.05623092  0.12441321  0.15618579 -0.10845048
  0.19041681  0.04404324  0.04022989  0.14824662  0.0595033  -0.08556162
 -0.11301917 -0.09236342  0.

 -0.03095328  0.01428331  0.03373021  0.06092547]
lorre
[ 0.02578195 -0.05168074  0.06716207 -0.02988144  0.05437604  0.0346606
 -0.09305339 -0.02704231  0.02968723 -0.01879351  0.00790453 -0.12091418
  0.03613449 -0.05068119  0.01192892  0.04684352]
thirds
[ 0.05476967  0.04540567  0.01361454  0.03234142 -0.00961208  0.15568969
 -0.07441358 -0.03881874 -0.03453763 -0.04692716  0.01880984  0.05236162
  0.00876687 -0.04931882 -0.05580982  0.10202718]
fewer
[ 0.08740636  0.0261384   0.02441677 -0.08665416  0.13182212  0.10024665
  0.12764859 -0.19545558 -0.08366482 -0.04374567 -0.04390436  0.05964968
  0.12154299 -0.01477775  0.08546459  0.04649632]
reportedly
[ 0.03351978  0.03727286 -0.10294592  0.08328678  0.15868278 -0.1706722
  0.17415908 -0.16330057 -0.01217077  0.06140386  0.07761379  0.05535133
  0.01649598 -0.01853914 -0.00799463 -0.0436637 ]
manipulate
[-0.10302666  0.06648237 -0.08025943  0.08380654 -0.04094179 -0.05890056
  0.03759035  0.10548642  0.03041966  0.07255601 -0.04

 -2.2200316e-02 -8.4754653e-02  2.9555628e-02 -7.2450884e-02]
neglect
[-0.0158982  -0.10982989  0.00414438  0.1084532   0.10685602 -0.06141338
  0.175426   -0.06235252  0.09489953  0.00704987  0.0150187  -0.24483797
 -0.0431301  -0.0424488  -0.03046623 -0.101603  ]
nutty
[-0.04440593 -0.01135376 -0.1708238   0.02481167 -0.00969337 -0.12398636
  0.10123128  0.00064141  0.00763183  0.07333907  0.054466   -0.04881771
 -0.05565844  0.00415731  0.00658804 -0.06971875]
behalf
[-0.01616088 -0.01683803 -0.09400148  0.0149004   0.09497186  0.01519025
  0.10651072 -0.11692251  0.03657202 -0.05161716  0.01481351 -0.01419862
 -0.01311295  0.03185951 -0.03354159 -0.01766248]
delia
[ 0.10644045  0.02200985  0.10052501 -0.14160508 -0.08191798 -0.05096674
  0.01268465  0.00190122 -0.13668545 -0.07160494 -0.14237821  0.15658721
  0.09919545  0.15529302  0.00630181  0.15420908]
borderline
[-0.16220978 -0.01137123 -0.16708526  0.15021327  0.14938746  0.10095574
  0.08212215  0.00224387  0.10925177  0.085

  0.1742646   0.11311252  0.01646252  0.09115267]
forgivable
[ 0.08206198 -0.03630973  0.09601446 -0.01429494 -0.00589515  0.14546011
  0.04006222 -0.05311541 -0.04939187 -0.07796652 -0.00676106  0.0597551
  0.05751026  0.09823125 -0.02363759  0.01461316]
sheila
[ 0.01245156  0.01043469 -0.0491557   0.08235312  0.08015841 -0.05889034
  0.02000574 -0.14909613  0.02827533  0.04243743 -0.0840981  -0.00553494
 -0.05468576  0.01664483  0.02029936 -0.08699874]
viggo
[-0.06780412  0.18090095 -0.02181841 -0.0204126   0.16483434 -0.04424474
  0.1125081   0.04795109  0.05813628  0.0623729   0.14527775  0.08100457
 -0.02363332 -0.01586618  0.02284807  0.01884003]
celeste
[ 0.01196117 -0.02116985 -0.02083702 -0.00722135  0.0814967  -0.11285767
  0.05106128 -0.05491544 -0.02514056  0.05340065 -0.22219826 -0.05578636
  0.01066995  0.07752675 -0.10768624  0.03232542]
flea
[-0.08905341 -0.17149745 -0.04919877  0.07856515  0.01840106 -0.2142703
  0.12543856 -0.22649705  0.12597734  0.12207156  0.030584

  0.12118275 -0.0855501  -0.01551782  0.02181838]
voting
[-0.07808129  0.02142362 -0.08768398  0.00181312 -0.12785383  0.05003814
 -0.03112535  0.11234493  0.06107114  0.09005696 -0.12244324  0.00203795
 -0.03749527 -0.21641885  0.07338267 -0.05625036]
praying
[ 0.01636283  0.01192475  0.09788915 -0.14099367 -0.13462569 -0.03946645
 -0.15481617  0.07619967 -0.02789486 -0.08579219  0.17888738 -0.15788683
  0.01483362  0.05024289 -0.05046405  0.02381501]
endured
[ 0.09109812  0.0884451   0.01541829 -0.0045652  -0.12209131 -0.01693947
 -0.07803021  0.07645827  0.06056243 -0.06716081  0.06544654 -0.01932975
 -0.00438708  0.09791745 -0.08835552  0.00259652]
rocker
[ 0.03462787 -0.07082459  0.10495284 -0.07011143  0.10447175  0.01536747
  0.00442609 -0.10270806 -0.14035307 -0.01715495  0.02740059 -0.18072927
  0.02476914 -0.05589252 -0.09025545  0.04449693]
exaggeration
[ 0.02653486 -0.0114309   0.02158355 -0.09066039 -0.14580312 -0.12834562
 -0.10025343 -0.22693436  0.16431487 -0.07515716  

In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

**Note:** The following code just allows you to download the .tsv files if you're running this notebook in Colab.

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [88]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [966], [4], [1537], [1537], [4715], [], [790], [2019], [11], [2929], [2184], [], [790], [2019], [11], [579], [], [11], [579], [], [4], [1782], [4], [4517], [11], [2929], [1275], [], [], [2019], [1003], [2929], [966], [579], [790], []]
